In [37]:
# Reference webiste for the model: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import plotly.graph_objs as go

In [114]:
#Importing CSV
csvpath = Path("Resources/LQD.csv")
ETF = pd.read_csv(csvpath)
ETF.head()


,Date,open,high,low,close,volume
0,2011-05-02 00:00:00-04:00,110.05,110.43,109.96,110.31,1374460.0
1,2011-05-03 00:00:00-04:00,110.33,110.40,110.09,110.25,648982.0
2,2011-05-04 00:00:00-04:00,110.46,110.59,110.31,110.40,656364.0
3,2011-05-05 00:00:00-04:00,110.51,110.88,110.51,110.79,761126.0
4,2011-05-06 00:00:00-04:00,110.56,111.03,110.42,110.77,830286.0


In [115]:
# Removing all columns but close_price
ETF.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
ETF.head()

,Date,close
0,2011-05-02 00:00:00-04:00,110.31
1,2011-05-03 00:00:00-04:00,110.25
2,2011-05-04 00:00:00-04:00,110.40
3,2011-05-05 00:00:00-04:00,110.79
4,2011-05-06 00:00:00-04:00,110.77


In [116]:
#Checking For Nulls
ETF.isnull().sum()

Date     0
close    3
dtype: int64

In [117]:
#Establishing test and train data for Closing price as well as the Date
close_data = ETF['close'].values
close_data = close_data.reshape((-1,1))

In [118]:
split_percent = 0.7
split = int(split_percent*len(close_data))

In [119]:
close_train = close_data[:split]
close_test = close_data[split:]
close_train

array([[110.31 ],
       [110.25 ],
       [110.4  ],
       ...,
       [117.225],
       [117.16 ],
       [116.75 ]])

In [120]:
date_data = ETF['Date']
date_train = ETF['Date'][:split]
date_test = ETF['Date'][split:]
print(len(close_train))
print(len(close_test))

1710
733


In [121]:
# setting the lookback, shape and 
look_back = 45

shape = (look_back,1)
# Reformat the data set to dataset of windows over a period of time.
#TimeseriesGenerator(Data, Target, sequence_length, batch)

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [122]:
#Build LSTM Model
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape= shape,
))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')


In [123]:
# Show the model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [124]:
# Train the model
#used fit_generator because we are looking to generate new predicted data points 
num_epochs = 200
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/200
84/84 [==============================] - 3s 15ms/step - loss: 16976.5242: 0s - loss: 17057.5
Epoch 2/200
84/84 [==============================] - 1s 15ms/step - loss: 14681.4950
Epoch 3/200
84/84 [==============================] - 1s 16ms/step - loss: 13668.8805
Epoch 4/200
84/84 [==============================] - 1s 16ms/step - loss: 13749.2784
Epoch 5/200
84/84 [==============================] - 1s 17ms/step - loss: 13590.8284
Epoch 6/200
84/84 [==============================] - 2s 22ms/step - loss: 13663.0641
Epoch 7/200
84/84 [==============================] - 2s 19ms/step - loss: 13561.6958
Epoch 8/200
84/84 [==============================] - 2s 23ms/step - loss: 13679.3051
Epoch 9/200
84/84 [==============================] - 2s 20ms/step - loss: 13597.6917
Epoch 10/200
84/84 [==============================] - 1s 17ms/step - loss: 13657.9579
Epoch 11/200
84/84 [==============================] - 1s 16ms/step - loss: 13514.5684
Epoch 12/200
84/84 [=======================

84/84 [==============================] - 1s 14ms/step - loss: 12237.2513
Epoch 94/200
84/84 [==============================] - 1s 14ms/step - loss: 12251.9328
Epoch 95/200
84/84 [==============================] - 1s 14ms/step - loss: 12067.3750
Epoch 96/200
84/84 [==============================] - 1s 14ms/step - loss: 12080.4274
Epoch 97/200
84/84 [==============================] - 1s 15ms/step - loss: 12015.9942
Epoch 98/200
84/84 [==============================] - 1s 17ms/step - loss: 11932.1243
Epoch 99/200
84/84 [==============================] - 1s 15ms/step - loss: 12099.7143
Epoch 100/200
84/84 [==============================] - 1s 15ms/step - loss: 11878.4058
Epoch 101/200
84/84 [==============================] - 1s 15ms/step - loss: 11935.0961
Epoch 102/200
84/84 [==============================] - 1s 15ms/step - loss: 11825.9919
Epoch 103/200
84/84 [==============================] - 1s 15ms/step - loss: 11953.9065
Epoch 104/200
84/84 [==============================] - 1s 15ms/

84/84 [==============================] - 1s 16ms/step - loss: 3.1822
Epoch 190/200
84/84 [==============================] - 2s 19ms/step - loss: 2.2622
Epoch 191/200
84/84 [==============================] - 1s 17ms/step - loss: 2.1294
Epoch 192/200
84/84 [==============================] - 1s 16ms/step - loss: 1.6352
Epoch 193/200
84/84 [==============================] - 1s 15ms/step - loss: 2.1878
Epoch 194/200
84/84 [==============================] - 1s 15ms/step - loss: 1.9670
Epoch 195/200
84/84 [==============================] - 1s 15ms/step - loss: 2.0753
Epoch 196/200
84/84 [==============================] - 1s 15ms/step - loss: 1.6081
Epoch 197/200
84/84 [==============================] - 1s 15ms/step - loss: 2.0377
Epoch 198/200
84/84 [==============================] - 1s 15ms/step - loss: 1.7621
Epoch 199/200
84/84 [==============================] - 1s 15ms/step - loss: 3.4013
Epoch 200/200
84/84 [==============================] - 1s 15ms/step - loss: 2.0081: 0s - lo


In [126]:
#Make predictions for futre prices
prediction = model.predict_generator(test_generator)

#Test model predictions for accuracy through plottting
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

#PLot for training prices
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
# PLotting our predicted prices
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
#plotting the actual prices for the period of time that was predicted
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
#Plot features and labels
layout = go.Layout(
    title = "LQD",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
#Show plot
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [107]:
#Predicting future prices

close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = ETF['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 90
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [108]:
forecast

array([136.52      , 137.97097778, 137.97879028, 138.35874939,
       138.41221619, 138.57046509, 139.20671082, 139.88172913,
       139.54519653, 138.96661377, 139.71481323, 138.97793579,
       139.03683472, 139.24310303, 138.44517517, 139.42062378,
       139.50271606, 139.30079651, 139.4755249 , 139.20213318,
       139.11079407, 139.02345276, 138.72775269, 139.24452209,
       139.23374939, 139.66625977, 139.68258667, 139.58760071,
       139.66914368, 139.78924561, 139.2852478 , 138.87966919,
       136.69065857, 136.54949951, 137.19595337, 136.42169189,
       137.51881409, 138.89891052, 139.50741577, 138.96992493,
       138.67753601, 138.59986877, 138.67163086, 139.12112427,
       139.98709106, 140.47531128, 140.27642822, 141.11042786,
       140.69706726, 140.75404358, 141.33895874, 140.3861084 ,
       141.60910034, 141.7003479 , 141.40705872, 141.58436584,
       141.20413208, 141.13409424, 141.06826782, 140.6348877 ,
       141.44778442, 141.42771912, 141.74139404, 141.72

In [109]:
#making a dataframe
predicted_prices = pd.DataFrame({'Date': forecast_dates, 'LQD_close_price': list(forecast)}, columns=['Date', 'LQD_close_price'])
predicted_prices.head()

,Date,TLT_close_price
0,2021-01-13 00:00:00-05:00,136.520000
1,2021-01-14 00:00:00-05:00,137.970978
2,2021-01-15 00:00:00-05:00,137.978790
3,2021-01-16 00:00:00-05:00,138.358749
4,2021-01-17 00:00:00-05:00,138.412216


In [110]:
#export to excel
predicted_prices.to_csv(r'Resources/LQD_forecast.csv')

In [111]:
# Combine all forecasted prices into a single DF
LQDpath = Path("Resources/LQD_forecast.csv")
LQD = pd.read_csv(LQDpath)
#LQD.set_index(LQD['Date'], inplace=True)

PZApath = Path("Resources/PZA_forecast.csv")
PZA = pd.read_csv(PZApath)
PZA = PZA['PZA_close_price']
#PZA.set_index(PZA['Date'], inplace=True)

USOpath = Path("Resources/USO_forecast.csv")
USO = pd.read_csv(USOpath)
USO = USO['USO_close_price']
#USO.set_index(USO['Date'], inplace=True)

TLTpath = Path("Resources/TLT_forecast.csv")
TLT = pd.read_csv(TLTpath)
TLT = TLT['TLT_close_price']
#TLT.set_index(TLT['Date'], inplace=True)

QQQpath = Path("Resources/QQQ_forecast.csv")
QQQ = pd.read_csv(QQQpath)
QQQ = QQQ['QQQ_close_price']
#QQQ.set_index(QQQ['Date'], inplace=True)

VMBSpath = Path("Resources/VMBS_forecast.csv")
VMBS = pd.read_csv(VMBSpath)
VMBS = VMBS['VMBS_close_price']
#VMBS.set_index(VMBS['Date'], inplace=True)

VNQpath = Path("Resources/VNQ_forecast.csv")
VNQ = pd.read_csv(VNQpath)
VNQ = VNQ['VNQ_close_price']

LQD.head()

0    136.520000
1    137.970978
2    137.978790
3    138.358749
4    138.412216
Name: TLT_close_price, dtype: float64

In [112]:
#Concatenate and clean the data
forecast_prices = pd.concat([LQD, PZA, USO, TLT, QQQ, VMBS, VNQ], axis='columns', join='inner')
forecast_prices = forecast_prices.drop(columns=["Unnamed: 0"])
forecast_prices.head()

,Date,LQD_close_price,PZA_close_price,USO_close_price,TLT_close_price,QQQ_close_price,VMBS_close_price,VNQ_close_price
0,2021-01-13 00:00:00-05:00,136.520000,27.090000,35.760000,136.520000,130.800000,54.010000,84.310000
1,2021-01-14 00:00:00-05:00,137.277298,27.168526,35.511150,137.970978,126.370728,54.105686,81.114426
2,2021-01-15 00:00:00-05:00,138.050400,27.249393,35.356846,137.978790,131.667343,54.160480,80.127251
3,2021-01-16 00:00:00-05:00,138.771729,27.327137,35.131569,138.358749,131.501678,54.233402,80.206505
4,2021-01-17 00:00:00-05:00,139.553513,27.401978,34.907616,138.412216,135.907135,54.292553,80.162140


In [113]:
#EXPORT EXCEL
forecast_prices.to_csv(r'Resources/forecast_prices.csv')